<a href="https://colab.research.google.com/github/SeoyoungOhMe/skt-fly-ai-challenger-class/blob/master/07_svm_multi_wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 랜덤포레스트 다중분류
- 와인품질 데이터셋

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### 1.데이터준비

In [ ]:
!wget https://raw.githubusercontent.com/devdio/datasets/main/winequalityN.csv

--2023-12-28 03:25:59--  https://raw.githubusercontent.com/devdio/datasets/main/winequalityN.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 390376 (381K) [text/plain]
Saving to: ‘winequalityN.csv’

winequalityN.csv    100%[===================>] 381.23K  --.-KB/s    in 0.04s   

2023-12-28 03:25:59 (9.94 MB/s) - ‘winequalityN.csv’ saved [390376/390376]



In [ ]:
wine = pd.read_csv('winequalityN.csv')
wine.shape

(6497, 13)

In [ ]:
df = wine.copy()
df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [ ]:
df.columns

Index(['type', 'fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality'],
      dtype='object')

In [ ]:
df.columns = [ c.replace(' ', '_') for c in df.columns ] # 칼럼 이름의 공백을 언더바로 대체
df.head()

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed_acidity         6487 non-null   float64
 2   volatile_acidity      6489 non-null   float64
 3   citric_acid           6494 non-null   float64
 4   residual_sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free_sulfur_dioxide   6497 non-null   float64
 7   total_sulfur_dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


In [ ]:
df.isna().sum(axis=0)

type                     0
fixed_acidity           10
volatile_acidity         8
citric_acid              3
residual_sugar           2
chlorides                2
free_sulfur_dioxide      0
total_sulfur_dioxide     0
density                  0
pH                       9
sulphates                4
alcohol                  0
quality                  0
dtype: int64

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed_acidity,6487.0,7.216579,1.296750,3.80000,6.40000,7.00000,7.70000,15.90000
volatile_acidity,6489.0,0.339691,0.164649,0.08000,0.23000,0.29000,0.40000,1.58000
citric_acid,6494.0,0.318722,0.145265,0.00000,0.25000,0.31000,0.39000,1.66000
residual_sugar,6495.0,5.444326,4.758125,0.60000,1.80000,3.00000,8.10000,65.80000
chlorides,6495.0,0.056042,0.035036,0.00900,0.03800,0.04700,0.06500,0.61100
free_sulfur_dioxide,6497.0,30.525319,17.749400,1.00000,17.00000,29.00000,41.00000,289.00000
total_sulfur_dioxide,6497.0,115.744574,56.521855,6.00000,77.00000,118.00000,156.00000,440.00000
density,6497.0,0.994697,0.002999,0.98711,0.99234,0.99489,0.99699,1.03898
pH,6488.0,3.218395,0.160748,2.72000,3.11000,3.21000,3.32000,4.01000
sulphates,6493.0,0.531215,0.148814,0.22000,0.43000,0.51000,0.60000,2.00000


In [ ]:
df['type'].value_counts()

white    4898
red      1599
Name: type, dtype: int64

In [ ]:
df['quality'].value_counts().sort_index()

3      30
4     216
5    2138
6    2836
7    1079
8     193
9       5
Name: quality, dtype: int64

In [ ]:
df['quality'] - 3

0       3
1       3
2       3
3       3
4       3
       ..
6492    2
6493    3
6494    3
6495    2
6496    3
Name: quality, Length: 6497, dtype: int64

### 2.테스트데이터 분리

In [ ]:
df.head()

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [ ]:
X = df.iloc[:, 1:-1]
X.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9


In [ ]:
y = df.iloc[:, -1]
y.head()

0    6
1    6
2    6
3    6
4    6
Name: quality, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=0)

In [ ]:
# 잘 분리되었는지 확인
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(5197, 11) (1300, 11)
(5197,) (1300,)


In [ ]:
y_train.value_counts()

6    2269
5    1710
7     863
4     173
8     154
3      24
9       4
Name: quality, dtype: int64

In [ ]:
y_test.value_counts()

6    567
5    428
7    216
4     43
8     39
3      6
9      1
Name: quality, dtype: int64

### 3.전처리
- 결측치 삭제
- 레드, 화이트 인코딩(0, 1)
- 스케일링

- 넘파이배열로 변환

In [ ]:
X_train.isna().sum(axis=0)

fixed_acidity           7
volatile_acidity        5
citric_acid             3
residual_sugar          2
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      8
sulphates               4
alcohol                 0
dtype: int64

In [ ]:
X_train.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
4209,6.6,0.23,0.32,1.7,0.024,26.0,102.0,0.99084,3.29,0.60,11.8
4754,6.4,0.19,0.35,10.2,0.043,40.0,106.0,0.99632,3.16,0.50,9.7
1729,7.5,0.22,0.33,6.7,0.036,45.0,138.0,0.99390,3.20,0.68,11.4
5196,7.2,0.65,0.02,2.3,0.094,5.0,31.0,0.99930,3.67,0.80,9.7
5623,9.0,0.66,0.17,3.0,0.077,5.0,13.0,0.99760,3.29,0.55,10.4


In [ ]:
X_train = X_train.dropna()

In [ ]:
# X_train 결측치 제거 후, y_train도 삭제

y_train = y_train[X_train.index]
y_train.shape

(5171,)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
#scaler.fit(X_train)
#scaler.transform(scaler)
X_train_s = scaler.fit_transform(X_train)
X_train_s # numpy array 형태로 출력

array([[-4.75055883e-01, -6.67636832e-01, -7.75036607e-04, ...,
         4.48211570e-01,  4.56526781e-01,  1.09866654e+00],
       [-6.27783798e-01, -9.11787399e-01,  2.06520530e-01, ...,
        -3.61259345e-01, -2.05011676e-01, -6.60334459e-01],
       [ 2.12219736e-01, -7.28674473e-01,  6.83234857e-02, ...,
        -1.12191371e-01,  9.85757546e-01,  7.63618729e-01],
       ...,
       [-1.46778733e+00, -1.09490032e+00,  4.82914620e-01, ...,
         2.19168739e+00,  3.90372935e-01,  9.35231111e-02],
       [-7.04147756e-01, -1.09490032e+00,  5.52013142e-01, ...,
         1.38221647e+00, -6.55013904e-03,  7.63618729e-01],
       [ 1.35855778e-01,  1.71283120e+00, -1.86643514e+00, ...,
         1.25768249e+00,  2.58065244e-01,  8.47380681e-01]])

In [ ]:
y_train = y_train.values
type(y_train)

numpy.ndarray

In [ ]:
X_train_s = X_train_s.values

AttributeError: ignored

In [ ]:
type(X_train_s)

In [ ]:
X_train_s.shape, y_train.shape

((5171, 11), (5171,))

### 4.학습

- 베이스 모델

In [ ]:
from sklearn.svm import SVC

clf = SVC(random_state = 42)
clf = clf.fit(X_train_s, y_train)

In [ ]:
X_test.columns

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')

In [ ]:
X_test = X_test.dropna()

In [ ]:
# X_train 결측치 제거 후, y_train도 삭제

y_test = y_test[X_test.index]
y_test.shape

(1292,)

In [ ]:
y_test.value_counts()


6    564
5    425
7    215
4     42
8     39
3      6
9      1
Name: quality, dtype: int64

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
#scaler.fit(X_train)
#scaler.transform(scaler)
X_test_s = scaler.fit_transform(X_test)
X_test_s # numpy array 형태로 출력

array([[-8.80510299e-01,  1.13004966e+00, -1.58538127e+00, ...,
         2.36593690e+00,  1.06256780e+00, -1.77166199e-01],
       [-3.20382625e-01,  1.90496195e+00, -1.65332543e+00, ...,
         4.37911819e-01,  4.89028672e-01, -1.01780937e+00],
       [-9.60528538e-01,  3.25333056e-01, -2.12893455e+00, ...,
         9.35466678e-01, -1.08820394e+00,  5.79412656e-01],
       ...,
       [-8.00492060e-01, -6.58209465e-01, -5.66218864e-01, ...,
        -1.05475276e+00, -1.80512785e+00,  7.50267525e-02],
       [ 2.16018279e+00,  1.72613604e+00, -4.98274704e-01, ...,
        -8.05975330e-01,  2.73951498e-01, -9.33745055e-01],
       [-3.09668108e-04, -8.37035379e-01, -2.26655828e-02, ...,
         1.12204975e+00, -8.01434371e-01, -5.13423468e-01]])

In [ ]:
y_pred = clf.predict(X_test_s)
y_pred

array([5, 5, 5, ..., 6, 5, 6])

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def print_metrics(y_test, y_pred):
  print(f'accuracy : {accuracy_score(y_test, y_pred)}')
  print(f'recall : {recall_score(y_test, y_pred, average="macro")}')
  print(f'precision : {precision_score(y_test, y_pred, average="macro")}')
  print(f'f1 : {f1_score(y_test, y_pred, average="macro")}')

In [ ]:
print("SVM")
print_metrics(y_test, y_pred)

SVM
accuracy : 0.5727554179566563
recall : 0.23052450668540553
precision : 0.25112300917170993
f1 : 0.22782662468858658
